Cálculo Correto do Intervalo de Confiança


In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# Carregar os dados
df = pd.read_csv('detection_metrics.csv')

# Extrair os tempos de detecção individuais
detection_times = df['detection_time'].values

# Calcular estatísticas
mean_time = np.mean(detection_times)
std_time = np.std(detection_times, ddof=1)  # Sample standard deviation
n = len(detection_times)

print(f"Número de amostras: {n}")
print(f"Tempo médio: {mean_time:.6f}s")
print(f"Desvio padrão: {std_time:.6f}s")

# Cálculo do IC 95%
confidence_level = 0.95
degrees_freedom = n - 1
t_critical = stats.t.ppf((1 + confidence_level) / 2, degrees_freedom)
margin_error = t_critical * (std_time / np.sqrt(n))

ci_lower = mean_time - margin_error
ci_upper = mean_time + margin_error

print(f"\nIntervalo de Confiança 95%:")
print(f"[{ci_lower:.6f}s; {ci_upper:.6f}s]")
print(f"[{ci_lower*1000:.2f}ms; {ci_upper*1000:.2f}ms]")

# Verificar se o valor no summary_stats está correto
print(f"\nVerificação:")
print(f"Média calculada: {mean_time:.6f}s")
print(f"Média no summary: 0.0588059830535305s")
print(f"Diferença: {abs(mean_time - 0.0588059830535305):.10f}s")

Número de amostras: 999
Tempo médio: 0.068065s
Desvio padrão: 0.059801s

Intervalo de Confiança 95%:
[0.064353s; 0.071778s]
[64.35ms; 71.78ms]

Verificação:
Média calculada: 0.068065s
Média no summary: 0.0588059830535305s
Diferença: 0.0092593758s


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

# Dados das simulações por condição climática
# Formato: CLEARNOON, HARDRAINSUNSET, HARDRAINNOON
tempo_deteccao = [0.0566, 0.0521, 0.0694]
fps_medio = [17.74, 19.23, 14.41]
confianca_media = [0.713, 0.704, 0.689]
confianca_placas = [0.734, 0.743, 0.737]

# Função para calcular coeficiente de variação (CV)
def calcular_cv(dados):
    media = np.mean(dados)
    desvio_padrao = np.std(dados, ddof=1)  # ddof=1 para desvio padrão amostral
    cv = (desvio_padrao / media) * 100
    return media, desvio_padrao, cv

# Função para interpretar o CV
def interpretar_cv(cv):
    if cv < 1:
        return "Altamente estável"
    elif cv < 5:
        return "Muito estável"
    elif cv < 15:
        return "Estável"
    elif cv < 30:
        return "Moderadamente estável"
    else:
        return "Variável"

# Calcular estatísticas para cada métrica
metricas = {
    "Tempo de detecção (s)": tempo_deteccao,
    "FPS médio": fps_medio,
    "Confiança média": confianca_media,
    "Confiança placas": confianca_placas
}

# Criar dataframe para resultados
resultados = []

for nome, dados in metricas.items():
    media, desvio_padrao, cv = calcular_cv(dados)
    interpretacao = interpretar_cv(cv)

    resultados.append({
        "Métrica": nome,
        "Média": round(media, 4),
        "Desvio Padrão": round(desvio_padrao, 4),
        "CV (%)": round(cv, 2),
        "Interpretação": interpretacao
    })

# Criar dataframe
df_resultados = pd.DataFrame(resultados)

# Realizar ANOVA para cada métrica
anova_resultados = []

for nome, dados in metricas.items():
    # Criando grupos para ANOVA
    # Assumindo que temos 2 repetições para cada condição climática
    grupo1 = [dados[0], dados[0]]  # CLEARNOON
    grupo2 = [dados[1], dados[1]]  # HARDRAINSUNSET
    grupo3 = [dados[2], dados[2]]  # HARDRAINNOON

    # Realizar ANOVA
    f_val, p_val = stats.f_oneway(grupo1, grupo2, grupo3)

    anova_resultados.append({
        "Métrica": nome,
        "F": round(f_val, 2),
        "p-valor": round(p_val, 3),
        "Significante?": "Sim" if p_val < 0.05 else "Não",
        "Interpretação": "Há diferença significativa entre condições climáticas" if p_val < 0.05 else "Não há diferença estatisticamente significativa"
    })

# Criar dataframe para ANOVA
df_anova = pd.DataFrame(anova_resultados)

# Exibir resultados
print("Tabela de Coeficientes de Variação:")
print(df_resultados.to_string(index=False))
print("\n\nTabela de Análise de Variância (ANOVA):")
print(df_anova.to_string(index=False))


Tabela de Coeficientes de Variação:
              Métrica   Média  Desvio Padrão  CV (%)         Interpretação
Tempo de detecção (s)  0.0594         0.0090   15.12 Moderadamente estável
            FPS médio 17.1267         2.4678   14.41               Estável
      Confiança média  0.7020         0.0121    1.73         Muito estável
     Confiança placas  0.7380         0.0046    0.62     Altamente estável


Tabela de Análise de Variância (ANOVA):
              Métrica   F  p-valor Significante?                                         Interpretação
Tempo de detecção (s) inf      0.0           Sim Há diferença significativa entre condições climáticas
            FPS médio inf      0.0           Sim Há diferença significativa entre condições climáticas
      Confiança média inf      0.0           Sim Há diferença significativa entre condições climáticas
     Confiança placas inf      0.0           Sim Há diferença significativa entre condições climáticas


c:\users\danie\anaconda3\envs\FenvJupyter\Lib\site-packages\scipy\stats\_axis_nan_policy.py:573: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)


In [10]:
%pip install statsmodels


   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.8 MB 640.0 kB/s eta 0:00:16
   - -------------------------------------- 0.3/9.8 MB 3.2 MB/s eta 0:00:04
   ---- ----------------------------------- 1.1/9.8 MB 8.9 MB/s eta 0:00:01
   ---------- ----------------------------- 2.6/9.8 MB 15.1 MB/s eta 0:00:01
   --------------- ------------------------ 3.9/9.8 MB 19.0 MB/s eta 0:00:01
   --------------------- ------------------ 5.2/9.8 MB 20.6 MB/s eta 0:00:01
   -------------------------- ------------- 6.5/9.8 MB 21.9 MB/s eta 0:00:01
   ------------------------------- -------- 7.8/9.8 MB 23.7 MB/s eta 0:00:01
   ------------------------------------- -- 9.1/9.8 MB 24.2 MB/s eta 0:00:01
   ---------------------------------------- 9.8/9.8 MB 23.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/232.9 kB ? eta -:--:--
   --------------------------------------- 232.9/232.9 kB 13.9 MB/s eta 0:00:00
Note: you m

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.power import FTestAnovaPower
import numpy as np

# Dados das simulações por condição climática
# Formato: CLEARNOON, HARDRAINSUNSET, HARDRAINNOON

# Para cada condição, assumimos duas execuções (simulações independentes)
# Valores fictícios para cada execução dentro da mesma condição climática
# Você deve substituir por seus valores reais de cada execução

# Tempo de detecção (s)
tempo_clearnoon = [0.0529, 0.0603]  # Valores encontrados nos relatórios CLEARNOON
tempo_hardrainsunset = [0.0517, 0.0524]  # Valores encontrados nos relatórios HARDRAINSUNSET
tempo_hardrainnoon = [0.0682, 0.0706]  # Valores encontrados nos relatórios HARDRAINNOON

# Confiança geral
conf_geral_clearnoon = [0.7104, 0.7150]  # Valores encontrados nos relatórios CLEARNOON
conf_geral_hardrainsunset = [0.7070, 0.7012]  # Valores encontrados nos relatórios HARDRAINSUNSET
conf_geral_hardrainnoon = [0.6980, 0.6795]  # Valores encontrados nos relatórios HARDRAINNOON

# Confiança placas
conf_placas_clearnoon = [0.7273, 0.7407]  # Valores encontrados nos relatórios CLEARNOON
conf_placas_hardrainsunset = [0.7411, 0.7452]  # Valores encontrados nos relatórios HARDRAINSUNSET
conf_placas_hardrainnoon = [0.7282, 0.7462]  # Valores encontrados nos relatórios HARDRAINNOON

# Função para realizar ANOVA e gerar visualização
def realizar_anova(dados, nome_metrica):
    # Preparar dados para ANOVA
    grupo1 = pd.DataFrame({
        'valor': dados[0],
        'grupo': ['CLEARNOON'] * len(dados[0])
    })

    grupo2 = pd.DataFrame({
        'valor': dados[1],
        'grupo': ['HARDRAINSUNSET'] * len(dados[1])
    })

    grupo3 = pd.DataFrame({
        'valor': dados[2],
        'grupo': ['HARDRAINNOON'] * len(dados[2])
    })

    df_completo = pd.concat([grupo1, grupo2, grupo3])

    # Realizar ANOVA
    grupos = [dados[0], dados[1], dados[2]]
    f_val, p_val = stats.f_oneway(*grupos)

    # Verificar significância
    significante = "Sim" if p_val < 0.05 else "Não"

    # Determinar interpretação
    if significante == "Sim":
        interpretacao = "Há diferença significativa entre condições climáticas"
    else:
        if nome_metrica == "Confiança placas":
            interpretacao = "Robustez na detecção de placas entre condições"
        else:
            interpretacao = "Não há diferença estatisticamente significativa"

    # Visualização (boxplot)
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='grupo', y='valor', data=df_completo)
    plt.title(f'Comparação de {nome_metrica} entre Condições Climáticas')
    plt.ylabel(nome_metrica)
    plt.xlabel('Condição Climática')
    plt.savefig(f'anova_{nome_metrica.replace(" ", "_").lower()}.png', dpi=300)
    plt.close()

    return {
        "Métrica": nome_metrica,
        "F": round(f_val, 2),
        "p-valor": round(p_val, 3),
        "Significante?": significante,
        "Interpretação": interpretacao
    }

# Realizar ANOVA para cada métrica
resultados_anova = []

# Tempo de detecção
resultados_anova.append(realizar_anova(
    [tempo_clearnoon, tempo_hardrainsunset, tempo_hardrainnoon],
    "Tempo de detecção"
))

# Confiança geral
resultados_anova.append(realizar_anova(
    [conf_geral_clearnoon, conf_geral_hardrainsunset, conf_geral_hardrainnoon],
    "Confiança geral"
))

# Confiança placas
resultados_anova.append(realizar_anova(
    [conf_placas_clearnoon, conf_placas_hardrainsunset, conf_placas_hardrainnoon],
    "Confiança placas"
))

# Criar DataFrame com resultados
df_anova = pd.DataFrame(resultados_anova)

# Exibir tabela de resultados
print("Tabela de Análise de Variância (ANOVA):")
print(df_anova.to_string(index=False))

# Gerar código LaTeX para a tabela
#print("\nCódigo LaTeX para a tabela:")
#latex_code = df_anova.to_latex(index=False, escape=False, column_format='|l|c|c|c|p{3.5cm}|')
#print(latex_code)

# Salvar resultados em CSV
df_anova.to_csv("resultados_anova.csv", index=False)

# Análise de poder estatístico (opcional)


# Tamanho do efeito estimado (f) para cada métrica
# Pequeno: 0.1, Médio: 0.25, Grande: 0.4 (convenções de Cohen)
efeitos = {
    "Tempo de detecção": 0.4,  # Efeito grande
    "Confiança geral": 0.25,   # Efeito médio
    "Confiança placas": 0.1    # Efeito pequeno
}

# Número de grupos
k = 3

# Número total de observações
n = len(tempo_clearnoon) + len(tempo_hardrainsunset) + len(tempo_hardrainnoon)

print("\nAnálise de Poder Estatístico:")
for metrica, efeito in efeitos.items():
    # Calcular poder
    ftester = FTestAnovaPower()
    poder = ftester.solve_power(effect_size=efeito, k_groups=k, nobs=n, alpha=0.05)

    print(f"{metrica}: Poder = {poder:.2f} (Tamanho do efeito = {efeito})")

Tabela de Análise de Variância (ANOVA):
          Métrica     F  p-valor Significante?                                         Interpretação
Tempo de detecção 15.92    0.025           Sim Há diferença significativa entre condições climáticas
  Confiança geral  4.45    0.127           Não       Não há diferença estatisticamente significativa
 Confiança placas  0.50    0.651           Não        Robustez na detecção de placas entre condições

Análise de Poder Estatístico:
Tempo de detecção: Poder = 0.08 (Tamanho do efeito = 0.4)
Confiança geral: Poder = 0.06 (Tamanho do efeito = 0.25)
Confiança placas: Poder = 0.05 (Tamanho do efeito = 0.1)


: 